In [192]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import os
import copy
import seaborn as sns

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold



from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import warnings
warnings.filterwarnings('ignore')

In [193]:
import faiss
import numpy as np


class FaissKMeans:
    def __init__(self, n_clusters=8, n_init=10, max_iter=300,seed=42):
        self.n_clusters = n_clusters
        self.n_init = n_init
        self.max_iter = max_iter
        self.kmeans = None
        self.cluster_centers_ = None
        self.inertia_ = None
        self.seed = seed

    def fit(self, X):
        self.kmeans = faiss.Kmeans(d=X.shape[1],
                                   k=self.n_clusters,
                                   niter=self.max_iter,
                                   nredo=self.n_init,
                                   seed=self.seed)

        self.kmeans.train(X.astype(np.float32))
        self.cluster_centers_ = self.kmeans.centroids
        self.inertia_ = self.kmeans.obj[-1]

    def predict(self, X):
        return self.kmeans.index.search(X.astype(np.float32), 1)[1]

In [194]:
from sklearn.preprocessing import QuantileTransformer

In [195]:

data_dir = '../data/01_raw'
os.listdir(data_dir)

['train_targets_scored.csv',
 'sample_submission.csv',
 '.gitkeep',
 'train_drug.csv',
 'train_features.csv',
 'test_features.csv',
 'train_targets_nonscored.csv']

In [196]:
# Parameters
no_ctl = True
ncompo_genes = 600
ncompo_cells = 50

In [197]:
train_features = pd.read_csv(data_dir+'/train_features.csv')
train_targets_scored = pd.read_csv(data_dir+'/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv(data_dir+'/train_targets_nonscored.csv')

test_features = pd.read_csv(data_dir+'/test_features.csv')
sample_submission = pd.read_csv(data_dir+'/sample_submission.csv')
drug = pd.read_csv(data_dir+'/train_drug.csv')

In [198]:
targets_scored = train_targets_scored.columns[1:]
scored = train_targets_scored.merge(drug, on='sig_id', how='left') 

In [199]:
if no_ctl:
    # cp_type == ctl_vehicle
    print("not_ctl")
    train_features = train_features[train_features["cp_type"] != "ctl_vehicle"].reset_index(drop=True)
    test_features = test_features[test_features["cp_type"] != "ctl_vehicle"].reset_index(drop=True)
    train_targets_scored = train_targets_scored.iloc[train_features.index]
    train_targets_nonscored = train_targets_nonscored.iloc[train_features.index]
    train_features.reset_index(drop = True, inplace = True)
    train_features.reset_index(drop = True, inplace = True)
    train_targets_scored.reset_index(drop = True, inplace = True)
    train_targets_nonscored.reset_index(drop = True, inplace = True)

not_ctl


In [200]:
# #Indiquer si valeur dans le range max, min

# import seaborn as sns
# data = pd.concat([train_features,test_features],axis=0)

# sns.distplot(data[data["cp_type"] == "ctl_vehicle"]["c-4"],label="normal")

# sns.distplot(data[data["cp_type"] == "trt_cp"]["c-4"],label="treated")
# plt.legend()
# plt.show()

In [201]:
# sum_targets = train_targets_scored[[c for c in train_targets_scored.columns if (c != "sig_id")]].sum().reset_index()

In [202]:
# sum_targets["index"] = sum_targets["index"].apply(lambda x : x.replace("_inhibitor",""))
# sum_targets["index"] = sum_targets["index"].apply(lambda x : x.replace("_activator",""))
# sum_targets["index"] = sum_targets["index"].apply(lambda x : x.replace("_agonist",""))
# sum_targets["index"] = sum_targets["index"].apply(lambda x : x.replace("_antagonist",""))

In [203]:
GENES = [col for col in train_features.columns if col.startswith('g-')]
CELLS = [col for col in train_features.columns if col.startswith('c-')]

In [204]:
# from scipy import stats

# train_features[GENES].apply(lambda x : stats.moment(x,moment=5),axis=1)

In [205]:
seed=42

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed)

In [206]:
#RankGauss

for col in (GENES + CELLS):
    transformer = QuantileTransformer(n_quantiles=100,random_state=seed, output_distribution="normal")
    vec_len = len(train_features[col].values)
    vec_len_test = len(test_features[col].values)
    raw_vec = train_features[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train_features[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_features[col] = transformer.transform(test_features[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

In [207]:
# GENES
n_comp = ncompo_genes 

data = pd.concat([pd.DataFrame(train_features[GENES]), pd.DataFrame(test_features[GENES])])
data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data[GENES]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_G-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_G-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(GENES))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [208]:
#CELLS
n_comp = ncompo_cells

data = pd.concat([pd.DataFrame(train_features[CELLS]), pd.DataFrame(test_features[CELLS])])
data2 = (PCA(n_components=n_comp, random_state=42).fit_transform(data[CELLS]))
train2 = data2[:train_features.shape[0]]; test2 = data2[-test_features.shape[0]:]

train2 = pd.DataFrame(train2, columns=[f'pca_C-{i}' for i in range(n_comp)])
test2 = pd.DataFrame(test2, columns=[f'pca_C-{i}' for i in range(n_comp)])

# drop_cols = [f'c-{i}' for i in range(n_comp,len(CELLS))]
train_features = pd.concat((train_features, train2), axis=1)
test_features = pd.concat((test_features, test2), axis=1)

In [209]:
train_features.shape

(21948, 1526)

In [210]:
from scipy import stats

def fe_stats(train, test):
    
    features_g = GENES
    features_c = CELLS
    
    for df in train, test:
        df['g_sum'] = df[features_g].sum(axis = 1) ## <==
        # df['g_mean'] = df[features_g].mean(axis = 1)
        df['g_std'] = df[features_g].std(axis = 1) ## <==
        # df['g_kurt'] = df[features_g].kurtosis(axis = 1)
        #df['g_skew'] = df[features_g].skew(axis = 1)
        # df['g_q25'] = df[features_g].quantile(q=.25,axis = 1)
        # df['g_q50'] = df[features_g].quantile(q=.5,axis = 1)
        # df['g_q75'] = df[features_g].quantile(q=.75,axis = 1)
        #df['g_var'] = df[features_g].apply(axis=1,func=stats.variation)
        # df['g_mad'] = df[features_g].mad(axis = 1)


        df['c_sum'] = df[features_c].sum(axis = 1) ## <==
        # df['c_mean'] = df[features_c].mean(axis = 1)
        df['c_std'] = df[features_c].std(axis = 1) ## <==
        # df['c_kurt'] = df[features_c].kurtosis(axis = 1)
        #df['c_skew'] = df[features_c].skew(axis = 1)
        # df['c_q25'] = df[features_c].quantile(q=.25,axis = 1)
        # df['c_q50'] = df[features_c].quantile(q=.5,axis = 1)
        # df['c_q75'] = df[features_c].quantile(q=.75,axis = 1)
        # df['c_var'] = df[features_c].apply(axis=1,func=stats.variation)
        # df['c_mad'] = df[features_c].mad(axis = 1)



        df['gc_sum'] = df[features_g + features_c].sum(axis = 1) ## <==
        # df['gc_mean'] = df[features_g + features_c].mean(axis = 1)
        # df['gc_std'] = df[features_g + features_c].std(axis = 1)
        # df['gc_kurt'] = df[features_g + features_c].kurtosis(axis = 1)
        # df['gc_skew'] = df[features_g + features_c].skew(axis = 1)
        # df['gc_q25'] = df[features_g + features_c].quantile(q=.25,axis = 1)
        # df['gc_q50'] = df[features_g + features_c].quantile(q=.5,axis = 1)
        # df['gc_q75'] = df[features_g + features_c].quantile(q=.75,axis = 1)
        # df['gc_var'] = df[features_g + features_c].apply(axis=1,func=stats.variation)
        # df['gc_mad'] = df[features_g + features_c].mad(axis = 1)


        
    return train, test

train_features,test_features=fe_stats(train_features,test_features)

In [211]:
# train_features_gc = train_features[["sig_id"]+GENES+CELLS].copy()
# test_features_gc = test_features[["sig_id"]+GENES+CELLS].copy()

In [212]:
from sklearn.feature_selection import VarianceThreshold


var_thresh = VarianceThreshold(0.8)  #<-- Update
data = train_features.append(test_features)
data_transformed = var_thresh.fit_transform(data.iloc[:, 4:])

train_features_transformed = data_transformed[ : train_features.shape[0]]
test_features_transformed = data_transformed[-test_features.shape[0] : ]


train_features = pd.DataFrame(train_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                              columns=['sig_id','cp_type','cp_time','cp_dose'])

train_features = pd.concat([train_features, pd.DataFrame(train_features_transformed)], axis=1)


test_features = pd.DataFrame(test_features[['sig_id','cp_type','cp_time','cp_dose']].values.reshape(-1, 4),\
                             columns=['sig_id','cp_type','cp_time','cp_dose'])

test_features = pd.concat([test_features, pd.DataFrame(test_features_transformed)], axis=1)

train_features.shape

(21948, 1047)

In [213]:
# from sklearn.cluster import KMeans
# def fe_cluster2(train, test, n_clusters = 3, SEED = 42):
    

#     def create_cluster(train, test, n_clusters = n_clusters):
#         train_ = train.copy()
#         test_ = test.copy()
#         data = pd.concat([train_, test_], axis = 0)
#         kmeans = KMeans(n_clusters = n_clusters, random_state = SEED).fit(data[[c for c in data.columns if c not in ["sig_id","cp_type","cp_dose","cp_time"]]])
#         train['cluster'] = kmeans.labels_[:train.shape[0]]
#         test['cluster'] = kmeans.labels_[train.shape[0]:]
#         train = pd.get_dummies(train, columns = ['cluster'])
#         test = pd.get_dummies(test, columns = ['cluster'])
#         return train, test
    
#     train, test = create_cluster(train, test, n_clusters = n_clusters)
#     return train, test



# train_features,test_features=fe_cluster2(train_features,test_features)



In [214]:
# data = pd.concat([train1, test1], axis = 0)

# distortion = []
# for k in range(1,10):
#     kmeans = KMeans(n_clusters = k, random_state = 42).fit(data)
#     distortion += [kmeans.inertia_]


In [215]:
# plt.plot(range(1,10),distortion)

In [216]:
# train_features = train_features.merge(train_features_gc.loc[:,[col for col in train_features_gc.columns if col not in GENES+CELLS]],on="sig_id")
# test_features = test_features.merge(test_features_gc.loc[:,[col for col in test_features_gc.columns if col not in GENES+CELLS]],on="sig_id")

In [217]:
train = train_features.merge(train_targets_scored, on='sig_id')
train = train.merge(train_targets_nonscored, on='sig_id')
# train = train[train['cp_type']!='ctl_vehicle'].reset_index(drop=True)
# test = test_features[test_features['cp_type']!='ctl_vehicle'].reset_index(drop=True)

target_scored = train[train_targets_scored.columns]
target_nscored = train[train_targets_nonscored.columns]

In [218]:
train = train.drop('cp_type', axis=1)
test = test_features.drop('cp_type', axis=1)

In [219]:
target_scored_cols = target_scored.drop('sig_id', axis=1).columns.values.tolist()
target_nscored_cols = target_nscored.drop('sig_id', axis=1).columns.values.tolist()

In [220]:
# folds = train.copy()

# mskf = MultilabelStratifiedKFold(n_splits=7)

# for f, (t_idx, v_idx) in enumerate(mskf.split(X=train, y=target_scored)):
#     folds.loc[v_idx, 'kfold'] = int(f)

# folds['kfold'] = folds['kfold'].astype(int)
# folds

In [221]:
class TrainDataset:
    def __init__(self, features, targets_scored,targets_nscored):
        self.features = features
        self.targets_scored = targets_scored
        self.targets_nscored = targets_nscored
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y_scored' : torch.tensor(self.targets_scored[idx, :], dtype=torch.float),
            'y_nscored' : torch.tensor(self.targets_nscored[idx, :], dtype=torch.float)           
        }
        return dct

class ValidDataset:
    def __init__(self, features, targets_scored):
        self.features = features
        self.targets_scored = targets_scored
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float),
            'y_scored' : torch.tensor(self.targets_scored[idx, :], dtype=torch.float),          
        }
        return dct
    
class TestDataset:
    def __init__(self, features):
        self.features = features
        
    def __len__(self):
        return (self.features.shape[0])
    
    def __getitem__(self, idx):
        dct = {
            'x' : torch.tensor(self.features[idx, :], dtype=torch.float)
        }
        return dct

In [222]:
def train_fn(model, optimizer, scheduler, loss_fn, dataloader, device):
    model.train()
    final_loss = 0
    
    for data in dataloader:
        optimizer.zero_grad()
        inputs, targets1, targets2 = data['x'].to(device), data['y_scored'].to(device), data['y_nscored'].to(device)
#         print(inputs.shape)
        outputs1,outputs2 = model(inputs)
        loss1 = loss_fn(outputs1, targets1)
        loss2 = loss_fn(outputs2, targets2)
        loss = loss1 + loss2
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        final_loss += loss.item()
        
    final_loss /= len(dataloader)
    
    return final_loss


def valid_fn(model, loss_fn, dataloader, device):
    model.eval()
    final_loss = 0
    valid_preds = []
    
    for data in dataloader:
        inputs, targets = data['x'].to(device), data['y_scored'].to(device)
        outputs,_ = model(inputs)
        loss = loss_fn(outputs, targets)
        
        final_loss += loss.item()
        valid_preds.append(outputs.sigmoid().detach().cpu().numpy())
        

    final_loss /= len(dataloader)


    valid_preds = np.concatenate(valid_preds)
    
    return final_loss, valid_preds

def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for data in dataloader:
        inputs = data['x'].to(device)

        with torch.no_grad():
            outputs,_ = model(inputs)
        
        preds.append(outputs.sigmoid().detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds

In [223]:
import torch
from torch.nn.modules.loss import _WeightedLoss
import torch.nn.functional as F

class SmoothBCEwLogits(_WeightedLoss):
    def __init__(self, weight=None, reduction='mean', smoothing=0.0):
        super().__init__(weight=weight, reduction=reduction)
        self.smoothing = smoothing
        self.weight = weight
        self.reduction = reduction

    @staticmethod
    def _smooth(targets:torch.Tensor, n_labels:int, smoothing=0.0):
        assert 0 <= smoothing < 1
        with torch.no_grad():
            targets = targets * (1.0 - smoothing) + 0.5 * smoothing
        return targets

    def forward(self, inputs, targets):
        targets = SmoothBCEwLogits._smooth(targets, inputs.size(-1),
            self.smoothing)
        loss = F.binary_cross_entropy_with_logits(inputs, targets,self.weight)

        if  self.reduction == 'sum':
            loss = loss.sum()
        elif  self.reduction == 'mean':
            loss = loss.mean()

        return loss

In [224]:
class Model(nn.Module):
    def __init__(self, num_features, num_targets_scored,num_targets_nscored, hidden_size,dropout_rate):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size))
        self.activation1 = torch.nn.PReLU(num_parameters = hidden_size, init = 1.0)
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        self.activation2 = torch.nn.PReLU(num_parameters = hidden_size, init = 1.0)


        self.batch_norm2b = nn.BatchNorm1d(hidden_size)
        self.dropout2b = nn.Dropout(dropout_rate)
        self.dense2b = nn.utils.weight_norm(nn.Linear(hidden_size, hidden_size))
        self.activation2b = torch.nn.PReLU(num_parameters = hidden_size, init = 1.0)
        
        self.batch_norm3 = nn.BatchNorm1d(hidden_size)
        self.dropout3 = nn.Dropout(dropout_rate)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets_scored))
        self.dense4 = nn.utils.weight_norm(nn.Linear(hidden_size, num_targets_nscored))

    def init_bias(self,pos_scored_rate,pos_nscored_rate):
        self.dense3.bias.data = nn.Parameter(torch.tensor(pos_scored_rate, dtype=torch.float))
        self.dense4.bias.data = nn.Parameter(torch.tensor(pos_nscored_rate, dtype=torch.float))
    
    def recalibrate_layer(self, layer):
        if(torch.isnan(layer.weight_v).sum() > 0):
            print ('recalibrate layer.weight_v')
            layer.weight_v = torch.nn.Parameter(torch.where(torch.isnan(layer.weight_v), torch.zeros_like(layer.weight_v), layer.weight_v))
            layer.weight_v = torch.nn.Parameter(layer.weight_v + 1e-7)

        if(torch.isnan(layer.weight).sum() > 0):
            print ('recalibrate layer.weight')
            layer.weight = torch.where(torch.isnan(layer.weight), torch.zeros_like(layer.weight), layer.weight)
            layer.weight += 1e-7

    def forward(self, x):
        x = self.batch_norm1(x)
        self.recalibrate_layer(self.dense1)
        x = self.activation1(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        self.recalibrate_layer(self.dense2)
        x = self.activation2(self.dense2(x))

        x = self.batch_norm2b(x)
        x = self.dropout2b(x)
        self.recalibrate_layer(self.dense2b)
        x = self.activation2b(self.dense2b(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        self.recalibrate_layer(self.dense3)
        x1 = self.dense3(x)

        self.recalibrate_layer(self.dense4)
        x2 = self.dense4(x)
        return x1,x2
    
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1):
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        pred = pred.log_softmax(dim=self.dim)
        with torch.no_grad():
            # true_dist = pred.data.clone()
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

In [225]:
def process_data(data):
    data = pd.get_dummies(data, columns=['cp_time','cp_dose'])
    return data

In [226]:
# HyperParameters

DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
#EPOCHS = 35
EPOCHS = 300 #200
PATIENCE=40

BATCH_SIZE = 128
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
NFOLDS = 7           
EARLY_STOPPING_STEPS = PATIENCE+5
EARLY_STOP = True

#hidden_size=1500
hidden_size=1000
dropout_rate = 0.2619422201258426
#dropout_rate = 0.28


In [227]:
# LOCATE DRUGS
vc = scored.drug_id.value_counts()
vc1 = vc.loc[vc<=18].index.sort_values()
vc2 = vc.loc[vc>18].index.sort_values()

# STRATIFY DRUGS 18X OR LESS
dct1 = {}; dct2 = {}
skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, 
          random_state=seed)
tmp = scored.groupby('drug_id')[targets_scored].mean().loc[vc1]
for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets_scored])):
    dd = {k:fold for k in tmp.index[idxV].values}
    dct1.update(dd)

# STRATIFY DRUGS MORE THAN 18X
skf = MultilabelStratifiedKFold(n_splits=NFOLDS, shuffle=True, 
          random_state=seed)
tmp = scored.loc[scored.drug_id.isin(vc2)].reset_index(drop=True)
for fold,(idxT,idxV) in enumerate( skf.split(tmp,tmp[targets_scored])):
    dd = {k:fold for k in tmp.sig_id[idxV].values}
    dct2.update(dd)

# ASSIGN FOLDS
folds = train.merge(drug,on="sig_id")
folds['fold'] = folds.drug_id.map(dct1)
folds.loc[folds.fold.isna(),'fold'] =\
    folds.loc[folds.fold.isna(),'sig_id'].map(dct2)
folds.fold = folds.fold.astype('int8')

In [228]:
feature_cols = [c for c in process_data(train).columns if c not in (target_scored_cols + target_nscored_cols)]
feature_cols = [c for c in feature_cols if c not in ['fold','sig_id']]
len(feature_cols)

1048

In [229]:
num_features=len(feature_cols)
num_targets_scored=len(target_scored_cols)
num_targets_nscored=len(target_nscored_cols)

In [230]:
print(train.shape)
print(folds.shape)
print(test.shape)
print(target_scored.shape)
print(target_nscored.shape)
print(sample_submission.shape)

(20228, 1654)
(20228, 1656)
(3624, 1046)
(20228, 207)
(20228, 403)
(3982, 207)


In [231]:
def run_training(fold, seed):
    
    seed_everything(seed)
    
    train = process_data(folds)
    test_ = process_data(test)

    
    trn_idx = train[train['fold'] != fold].index
    val_idx = train[train['fold'] == fold].index
    
    train_df = train[train['fold'] != fold].reset_index(drop=True)
    valid_df = train[train['fold'] == fold].reset_index(drop=True)
    
    x_train, y_scored_train, y_nscored_train  = train_df[feature_cols].values, train_df[target_scored_cols].values, train_df[target_nscored_cols].values
    x_valid, y_scored_valid =  valid_df[feature_cols].values, valid_df[target_scored_cols].values
    
    train_dataset = TrainDataset(x_train, y_scored_train, y_nscored_train)
    valid_dataset = ValidDataset(x_valid, y_scored_valid)
    trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets_scored=num_targets_scored,
        num_targets_nscored=num_targets_nscored,
        hidden_size=hidden_size,
        dropout_rate=dropout_rate,
    )

    # model.init_bias(pos_scored_rate,pos_nscored_rate)

    model.to(DEVICE)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    scheduler = optim.lr_scheduler.OneCycleLR(optimizer=optimizer,pct_start=0.1, div_factor=1e4, final_div_factor=1e5,
                                              max_lr=1e-2, epochs=EPOCHS, steps_per_epoch=len(trainloader))


    #scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,mode = "min", patience = PATIENCE, min_lr = 1e-6, factor = 0.9)
    
    loss_fn = nn.BCEWithLogitsLoss()
    loss_tr = SmoothBCEwLogits(smoothing =0.001)
    
    early_stopping_steps = EARLY_STOPPING_STEPS
    early_step = 0
   
    oof = np.zeros((len(train), target_scored.iloc[:, 1:].shape[1]))
    best_loss = np.inf
    
    for epoch in range(EPOCHS):
        
        train_loss = train_fn(model, optimizer,scheduler, loss_tr, trainloader, DEVICE)
        valid_loss, valid_preds = valid_fn(model, loss_fn, validloader, DEVICE)
        print(f"FOLD: {fold}, EPOCH: {epoch}, train_loss: {train_loss}, valid_loss: {valid_loss}")
        #scheduler.step(valid_loss)
        
        if valid_loss < best_loss:
            
            best_loss = valid_loss
            oof[val_idx] = valid_preds
            torch.save(model.state_dict(), f"FOLD{fold}_.pth")
        
        elif(EARLY_STOP == True):
            
            early_step += 1
            if (early_step >= early_stopping_steps):
                break
            
    
    #--------------------- PREDICTION---------------------
    x_test = test_[feature_cols].values
    testdataset = TestDataset(x_test)
    testloader = torch.utils.data.DataLoader(testdataset, batch_size=BATCH_SIZE, shuffle=False)
    
    model = Model(
        num_features=num_features,
        num_targets_scored=num_targets_scored,
        num_targets_nscored=num_targets_nscored,
        hidden_size=hidden_size,
        dropout_rate=dropout_rate
    )
    
    model.load_state_dict(torch.load(f"FOLD{fold}_.pth"))
    model.to(DEVICE)
    
    predictions = np.zeros((len(test_), target_scored.iloc[:, 1:].shape[1]))
    predictions = inference_fn(model, testloader, DEVICE)
    
    return oof, predictions

In [232]:
def run_k_fold(NFOLDS, seed):
    oof = np.zeros((len(train), len(target_scored_cols)))
    predictions = np.zeros((len(test), len(target_scored_cols)))
    
    for fold in range(NFOLDS):
        oof_, pred_ = run_training(fold, seed)
        
        predictions += pred_ / NFOLDS
        oof += oof_
        
    return oof, predictions

In [233]:
# Averaging on multiple SEEDS

# SEED = [0,1,2,3,4,5,6] #<-- Update
SEED = [0]
oof = np.zeros((len(train), len(target_scored_cols)))
predictions = np.zeros((len(test), len(target_scored_cols)))

# mean_scored = np.mean(train[target_scored_cols].values,axis=0)
# mean_nscored = np.mean(train[target_nscored_cols].values,axis=0)
# pos_scored_rate = np.log(np.where(mean_scored==0, 1e-8, mean_scored))
# pos_nscored_rate = np.log(np.where(mean_nscored==0, 1e-8, mean_nscored))

for seed in SEED:
    
    oof_, predictions_ = run_k_fold(NFOLDS, seed)
    oof += oof_ / len(SEED)
    predictions += predictions_ / len(SEED)

train[target_scored_cols] = oof
test[target_scored_cols] = predictions

FOLD: 0, EPOCH: 0, train_loss: 0.7100230056573363, valid_loss: 0.06298278628484062
FOLD: 0, EPOCH: 1, train_loss: 0.07386666574679754, valid_loss: 0.0277830909775651
FOLD: 0, EPOCH: 2, train_loss: 0.04261070547406288, valid_loss: 0.023268655914327373
FOLD: 0, EPOCH: 3, train_loss: 0.036808370557778025, valid_loss: 0.021836809247084286
FOLD: 0, EPOCH: 4, train_loss: 0.03420384299448308, valid_loss: 0.02082375696171885
FOLD: 0, EPOCH: 5, train_loss: 0.032647419323706454, valid_loss: 0.019990439243290737
FOLD: 0, EPOCH: 6, train_loss: 0.032018078107605964, valid_loss: 0.02193804749328157
FOLD: 0, EPOCH: 7, train_loss: 0.031125270035665703, valid_loss: 0.019080265708591625
FOLD: 0, EPOCH: 8, train_loss: 0.030483116628602147, valid_loss: 0.01849356820078
FOLD: 0, EPOCH: 9, train_loss: 0.030317508478594178, valid_loss: 0.018446685422373856
FOLD: 0, EPOCH: 10, train_loss: 0.029899670673972544, valid_loss: 0.018465962989822678
FOLD: 0, EPOCH: 11, train_loss: 0.02987897298846613, valid_loss: 0.

KeyboardInterrupt: 

In [ ]:
valid_results = train_targets_scored.drop(columns=target_scored_cols).merge(train[['sig_id']+target_scored_cols], on='sig_id', how='left').fillna(0)



y_true = train_targets_scored[target_scored_cols].values
y_pred = valid_results[target_scored_cols].values

score = 0
for i in range(len(target_scored_cols)):
    score_ = log_loss(y_true[:, i], y_pred[:, i])
    score += score_ / target_scored.shape[1]
    
print("CV log_loss: ", score)

In [ ]:
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_true,y_pred))

In [ ]:
#Baseline
#CV log_loss:  0.015877615364984873 - 0.6541240486632599
#without kmeans CV log_loss: 0.015854058954568463 - 0.659868777535432
#epochs 35 - CV log_loss:  0.01585343762816286 - 0.6599438843390867
#epocjs 40 - CV log_loss:  0.01586852336393351 - 0.6557867961263701
#all stats - CV log_loss:  0.015855185527591735 -0.6570807318381957

In [ ]:
#auc .8073 -> .6541 

In [48]:
#CV log_loss:  0.014772181976560779 - hidden size 1500
#CV log loss:  0.014761207506656483 - hidden size 1600
#CV log_loss:  0.014757089674479135 - hidden size 1600 + recalibrate layers
#CV log_loss:  0.014798489345352649 - hidden size 1600 + recalibrate layers + 30epochs
#CV log_loss:  0.014993945755985132 - hidden size 1600 + recalibrate layers - batch size 256
#CV log_loss:  0.014925429807583115 - hidden size 1600 + recalibrate layers - batch size 256 + 50 epcohs
#CV log_loss:  0.01501367781064934  - hidden size 1600 + recalibrate layers - batch size 256 + 50 epcohs div_factor=1e4
#CV log_loss:  0.015038120295874759 - hidden size 1600 + recalibrate layers - batch size 256 + 50 epcohs pct_start=0.2
#CV log_loss:  0.014925429807583115 - hidden size 1600 + recalibrate layers - batch size 256 + 50 epcohs 2*lr
#CV log_loss:  0.014862676189325522 - hidden size 2000 + recalibrate layers - batch size 256 + 50 epcohs
#CV log_loss:  0.014805892283163887 - hidden size 2000 + recalibrate layers - batch size 256 + 70 epcohs



#CV log_loss:  0.014701033773812383 - hidden size 1000 + recalibrate layers + 30 epochs + 1layer + stats g35 c5
#CV log_loss:  0.01470633477903798  - hidden size 1000 + recalibrate layers + 30 epochs + 1layer + stats g15 c5

#CV log_loss:  0.014688195780301675 - hidden size 1000 + recalibrate layers + 30 epochs + 1layer + stats + varianceth 0.81

#CV log_loss:  0.01473922963978107 - hidden size 1000 + recalibrate layers + 40 epochs + 1layer
#CV log_loss:  0.014704245299661241 - hidden size 900 + recalibrate layers + 30 epochs + 1layer
#CV log_loss:  0.014700136730168381 - hidden size 1100 + recalibrate layers + 30 epochs + 1layer


#CV log_loss:  0.015298968285739336 - hidden size 1600 + recalibrate layers - batch size 64

#CV log_loss:  0.014708188641129298 - hidden size 1000 + recalibrate layers + 30 epochs
#CV log_loss:  0.014686009232339573 - hidden size 1000 + recalibrate layers + 30 epochs + 1layer
#CV log_loss:  0.014675118998674056 - hidden size 1000 + recalibrate layers + 30 epochs + 1layer + stats
#

#CV log_loss:  0.01468776733769557 without stats





#CV log_loss:  0.014751931843087755 baseline
#CV log_loss:  0.014746491073514958 baseline + std g,c
#CV log_loss:  0.014753663287033052 baseline + std g,c + median g,c
#CV log_loss:  0.014747499643338137 baseline + std g,c + sum g,c + kurt g,c
#CV log_loss:  0.014745229709145196 baseline + std g,c + sum g,c + mean g,c
#CV log_loss:  0.014765909985927743 baseline + std g,c + sum g,c + skew g,c
#CV log_loss:  0.01475305848080635  baseline + std g,c + sum g,c + g_var
#CV log_loss:  0.014738260008810762 baseline + std g,c + sum g,c
#CV log_loss:  0.014735805473683174 baseline + std g,c + sum g,c,gc
#CV log_loss:  0.014750251194250852 baseline + std g,c,gc + sum g,c,gc
#CV log_loss:  0.01474079804921367  baseline + std g,c + sum g,c,gc + variance 80(threshold)

#CV log_loss:  0.014704018818104764 baseline + std g,c + sum g,c,gc + variance 80(threshold) &PCA
#CV log loss:  0.014669995520015213 baseline + std g,c + sum g,c,gc + variance 80(threshold) &PCA + clusters3 - CV log_loss:  0.014520597500516905 /0.8153730747072218 (seeds)

#CV log_loss:  0.014566130099844471/0.815729267497772 avec prelu, 30 epochs


#CV log_loss:  0.014685491308574154 baseline + std g,c + sum g,c,gc + variance 80(threshold) &PCA + clusters5


#CV log_loss:  0.015099951953285232 baseline + PCA features
